<a href="https://colab.research.google.com/github/yagmurdenizlii/nlp/blob/main/w2v_and_glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install torchvision

In [ ]:
!pip install nltk

import nltk
nltk.download('punkt')

In [ ]:
import random
from sklearn.model_selection import train_test_split
import csv
import numpy as np
import statistics
import os
import re
import pandas as pd
import torch
from tensorflow.python.ops.numpy_ops import np_config
import os
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt

In [ ]:
artist_names = [    'Beyonce', 
            'BillieEilish', 
            'CharliePuth',
            'ColdPlay', 
            'Drake', 
            'DuaLipa',
            'EdSheeran',
            'JustinBieber',
            'KatyPerry',
            'LadyGaga',
            'Maroon5',
            'NickiMinaj',
            'Rihanna',
            'SelenaGomez',
            'TaylorSwift',
            ]

    
list_with_spaces = ['Beyonce', 
                    'Billie Eilish', 
                    'Charlie Puth',
                    'ColdPlay', 
                    'Drake', 
                    'Dua Lipa',
                    'Ed Sheeran',
                    'Justin Bieber',
                    'Katy Perry',
                    'Lady Gaga',
                    'Maroon 5',
                    'Nicki Minaj',
                    'Rihanna',
                    'Selena Gomez',
                    'Taylor Swift',
                    'beyoncé',
                    'verse',
                    'chris martin',
                    'sean paul',
                    'justin timberlake',
                    'beyoncè'
                    ]



In [ ]:

list_of_files = [artist + '.csv' for artist in artist_names]

all_lyrics = []

for file_path in list_of_files:
    df = pd.read_csv(file_path)
    df = df.dropna()

    text_list = df['Lyric'].values.tolist() 

    new_all_lyrics = []

    for line in text_list:
        new_line = line.replace("'", '')
        for name in list_with_spaces:
            new_line = new_line.replace(name.lower(), '')
        new_all_lyrics.append(new_line)

    for row in new_all_lyrics:
        if type(row) is str:
            row_list = row.split()
            nb_words = len(row_list)
            if nb_words > 200:
                nb_chunks = nb_words // 200 + 1
                arr = np.array_split(row_list, nb_chunks)

                for array in arr:
                    all_lyrics.append(' '.join(array) + '\n')

            elif nb_words > 20:
                all_lyrics.append(row)

print('fin')       

fin


In [ ]:
from torch.utils.data import Dataset, DataLoader


# dataset definition
class MyDataset(Dataset):
    # load the dataset
    def __init__(self, list):
        # store the inputs and outputs
        self.list = list
        
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.list)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.list[idx]]

In [ ]:
import torch.utils.data as data
from torchvision import datasets

# create the dataset
dataset = MyDataset(all_lyrics)
# select rows from the dataset
train, test = data.random_split(dataset, [len(all_lyrics)*9//10, len(all_lyrics)//10 + 1])
# create a data loader for train and test sets
train_dl = DataLoader(train, batch_size=32, shuffle=True)
test_dl = DataLoader(test, batch_size=1024, shuffle=False)

In [ ]:
import re  
import pandas as pd 
from time import time 
from collections import defaultdict  

import spacy 

import logging  
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
all_lines = []

all_words = [nltk.word_tokenize(sent) for sent in all_lyrics]

print(len(all_words))
print(len(all_words[0]))

from nltk.corpus import stopwords
nltk.download('stopwords')

all_words_final = []
for group in all_words:
  group_final = []
  for word in group:
    if word not in stopwords.words('english'):
      group_final.append(word)
  all_words_final.append(group_final)

print(len(all_words_final))
print(len(all_words_final[0]))



In [ ]:
from gensim.models import Word2Vec

word2vec = Word2Vec(all_words, min_count=2)

In [ ]:
vocabulary = word2vec.wv.vocab
print(vocabulary)

t = time()
word2vec.train(all_words, total_examples=word2vec.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
similar_words = word2vec.wv.most_similar('love')
similar_words

In [ ]:
word2vec.wv.most_similar(positive=["love", "amazing"], negative=["hate"], topn=3)


In [ ]:
import multiprocessing

from gensim.models import Word2Vec

In [ ]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=3)


t = time()
w2v_model.build_vocab(all_words_final, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

t = time()
w2v_model.train(all_words_final, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
w2v_model.wv.most_similar(positive=["love"])


In [ ]:
w2v_model.wv.most_similar(positive=["love", "amazing"], negative=["hate"], topn=3)


In [ ]:

! pip install glove-python-binary
import nltk 
nltk.download('stopwords')
nltk.download ('punkt')
nltk.download('wordnet')

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

all_tokens = []

for song in all_lyrics:
 words = word_tokenize(song)
 all_tokens.append(words)

In [ ]:

from nltk.corpus import stopwords
nltk.download('stopwords')

all_tokens_final = []
for group in all_tokens:
  groups = []
  for word in group:
    if word not in stopwords.words('english'):
      groups.append(word)
  all_tokens_final.append(groups)

print(len(all_tokens_final))
print(len(all_tokens_final[0]))

In [ ]:
from nltk import WordNetLemmatizer  
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')

wordnet_lemmatizer = WordNetLemmatizer() 

lemmatized = [] 

for sentence in all_tokens_final: 
  lemmatized_words = []
  for word in sentence:
    lemmatized_words.append(wordnet_lemmatizer.lemmatize(word)) 
  lemmatized.append(lemmatized_words)

In [ ]:
wordnet_lemmatizer.lemmatize('leaves')

In [ ]:
from glove import Corpus, Glove

corpus = Corpus() 


corpus.fit(lemmatized, window=10)


glove = Glove(no_components=5, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

In [ ]:
love = glove.word_vectors[glove.dictionary['love']]
hate =  glove.word_vectors[glove.dictionary['hate']]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([love], [hate])

In [ ]:
import torch
import torchtext

# The first time you run this will download a ~823MB file
glove2 = torchtext.vocab.GloVe(name="6B", dim=50)

In [ ]:
x = glove2['cat']
y = glove2['dog']

torch.norm(y - x)

torch.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0))

In [ ]:
a = glove2['pop']
b = glove2['music']

torch.norm(a - b)

torch.cosine_similarity(a.unsqueeze(0), b.unsqueeze(0))